Say we're interested in creating a Rubik's cube speedcubing timer with the following features:

- The mean of all the solves in the session, with the top and bottom 5% of times removed.
- The best average of $n$ consecutive solves in the collection, for $n=5,12,100$.

Statistics need to be recomputed on inserts and deletes, and we want inserts and deletes to run in at most $O(\log n)$ time.

Approach #1: Use a list. Insert by appending $O(1)$, delete by removing from the list $O(n)$ and recompute the statistics by re-sorting the list each time $O(n \log n)$.

Approach #2: Maintain a heap and an ordered list. Use the heap to compute the global average $O(\log n)$ and the ordered list to compute consecutive averages $O(n)$.

Approach #3: Use augmented binary search trees.

We will use two different binary search trees (or alternatively, a heap and a binary search tree). The first tree stores the times sorted, and will be use to compute the global mean with the top and bottom percentiles trimmed. To facilitate computing the top and bottom percentiles, we'll augment each node the binary search tree with the number of nodes in its subtree and the sum of all the nodes in its subtree.

The second binary search tree will store the entries in sorted order. It will be used to compute the statistics for consecutive solves in the collection. To enable this, we'll store two additional properties on each node: the average ending at this node, and the best average in the subtree rooted at this node.

In [40]:
import math
import random
import json


class Node:
    """ Augmented BST node. """
    def __init__(self, key):
        self.key = key
        
        self.left = None
        self.right = None
        self.parent = None

        self.height = 0
        self.sum = 0
        self.size = 1

        
def get_height(node):
    """ Returns the height of `node` - the longest path from this node to a leaf node, plus 1. """
    return node.height if node else 0


def get_sum(node):
    """ Returns the sum of all values in the subtree rooted at this node, minus the node itself. """
    return node.sum if node else 0


def get_size(node):
    """ Returns the count of nodes in the subtree rooted at this node, not including the node itself. """
    return node.size if node else 0


def get_balance(root):
    """ Returns the AVL balance factor of `root`. """
    return height(root.left) - height(root.right)


def get_min(root):
    """ Returns the smallest value in the tree rooted at `root`. """
    while root.left:
        root = root.left
        
    return root


def get_max(root):
    """ Returns the largest value in the tree rooted at `root`. """
    while root.right:
        root = root.right
        
    return root


def get_successor(node):
    """ Returns the successor of `node`, i.e. the first value with a key larger than node. """
    # If there are right children, return the smallest value in the right subtree.
    if node.right:
        return get_min(node.right)
    
    parent = node.parent
    
    # If not, go up the tree until we reach the root, or we come up through the left branch.
    while parent and node == parent.right:
        node = parent
        parent = node.parent
    
    return parent
    
    
def get_predecessor(node):
    """ Returns the predecessor of `node`, i.e. the first value with a key smaller than node. """
    if node.left:
        return get_max(node.left)
    
    parent = node.parent
    
    while parent and node == parent.left:
        node = parent
        parent = node.parent
    
    return parent


def get_index(node):
    """ Returns the index of node if viewing the tree as an array. Indices begin at 0. """
    # Traverse up the tree and count the number of nodes smaller than `node`.
    r = get_size(node.left)
    
    while node.parent:
        if node == node.parent.right:
            r += 1 + get_size(node.parent.left)
    
        node = node.parent
        
    return r
    

def update_data(y, z):
    """ Update metadata (height, sum, size) after an AVL rotation. """
    z.height = 1 + max(height(z.left), height(z.right))
    z.sum = z.key + get_sum(z.left) + get_sum(z.right)
    z.size = 1 + get_size(z.left) + get_size(z.right)

    y.height = 1 + max(height(y.left), height(y.right))
    y.sum = y.key + get_sum(y.left) + get_sum(y.right)
    y.size = 1 + get_size(y.left) + get_size(y.right)

    y.parent = z.parent
    z.parent = y
    
    
def right_rotate(z):
    """ AVL right rotation. """
    y = z.left
    z.left = y.right
    y.right = z
    update_data(y, z)
    return y
    

def left_rotate(z):
    """ AVL left rotation. """
    y = z.right
    z.right = y.left
    y.left = z
    update_data(y, z)
    return y


def rebalance(root, inserted_key):
    """ Rebalance the tree rooted at `root` using AVL. """
    root_balance = get_balance(root)

    if root_balance > 1:
        if inserted_key < root.left.key:
            return right_rotate(root)
        else:
            root.left = left_rotate(root.left)
            return right_rotate(root)
    elif root_balance < -1:
        if inserted_key >= root.right.key:
            return left_rotate(root)
        else:
            root.right = right_rotate(root.right)
            return left_rotate(root)

    return root

            
def insert(root, node):
    """ Insert `node` into the tree beginning at `root`. """
    if root is None:
        return node

    node.parent = root
    root.sum += node.key
    root.size += 1

    if node.key < root.key:
        root.left = insert(root.left, node)
    else:
        root.right = insert(root.right, node)
            
    root.height = 1 + max(height(root.left), height(root.right))

    root = rebalance(root, node.key)

    return root


def select(tree, i):
    """ Find the item at index `i` viewing the tree as an ordered array. """
    if not tree:
        return None
    
    l = tree.left.size if tree.left else 0

    if i == l:
        return tree
    
    if i < l:
        return select(tree.left, i)
    
    return select(tree.right, i - l - 1)


def get_cum_sum(tree, key):
    """ Get the cumulative sum of values less than or equal to `key`. """
    if not tree:
        return 0

    if tree.key == key:
        return tree.key + get_sum(tree.left)
    
    if tree.key > key:
        return get_cum_sum(tree.left, key)
    
    if tree.key < key:
        return tree.key + get_sum(tree.left) + get_cum_sum(tree.right, key)


def get_sum_range(tree, i, j):
    """ Sum all elements from index `i` to `j`. """
    return get_cum_sum(tree, select(tree, j).key) - get_cum_sum(tree, select(tree, i).key) + select(tree, i).key


def get_percentile(size, percentile = 5 / 100):
    """ Get the indices of elements to consider when removing the top and bottom percentile of elements. """
    return [
        math.floor(size * percentile) + 1,
        math.floor(size * (1 - percentile) - 1)
    ]


def get_average(node, size):
    """ Compute the average value of `node` and the previous `size` el """
    if index(node) >= size - 1:
        values = []

        for i in range(size):
            values.append(node.value)
            node = get_predecessor(node)
            
        values.sort()
        
        start, end = get_percentile(len(values))
        values = values[start:end + 1]
        average = sum(values) / len(values)

        return average
        

def update_average(node, count):
    prop = 'best_ao' + str(count)

    for i in range(count):
        average = get_average(node, count)

        if average:
            setattr(node, 'current_ao' + str(count), average)

            current_node = node

            while current_node and (not getattr(current_node, prop) or average < getattr(current_node, prop)):
                setattr(current_node, prop, average)
                current_node = current_node.parent

        node = get_successor(node)
        
        if node is None:
            break
    

def update_averages(node):
    update_average(node, 5)
    update_average(node, 12)
    update_average(node, 100)

    
class CubeResultStore:
    
    history_tree = None
    ordered_tree = None
    

    def insert(self, value):
        i = get_size(self.history_tree)
        history_node = Node(i)
        history_node.value = value
        history_node.best_ao5 = None
        history_node.best_ao12 = None
        history_node.best_ao100 = None
        self.history_tree = insert(self.history_tree, 
                                   history_node)
        update_averages(history_node)

        ordered_node = Node(value)
        self.ordered_tree = insert(self.ordered_tree, 
                                   ordered_node)
        

    def get_average(self):
        start, end = get_percentile(self.ordered_tree.size)
        return get_sum_range(self.ordered_tree, start, end) / (end - start + 1)


    def get_best_ao5(self):
        return self.history_tree.best_ao5

        
store = CubeResultStore()

for i in range(int(1e4)):
    store.insert(random.gauss(10, 2))

print('Global average:', store.get_average())
print('Best ao5:', store.get_best_ao5())
print('History tree height:', store.history_tree.height)
print('Ordered tree height:', store.ordered_tree.height)

Global average: 9.900014794742159
Best ao5: 5.940216964989848
History tree height: 14
Ordered tree height: 15
